## Process Data

This notebook takes all the raw files from the bronze volume and processes them into a single table. It could be optimized to use less ram, but does the trick.

In [1]:
from hydrate.process import read_nested_parquet_files
df = read_nested_parquet_files("../3w_dataset")

In [2]:
from hydrate.process import add_state_name, clean_data
df = df.pipe(clean_data).pipe(add_state_name)

In [16]:
from hydrate.utils import get_config_path
config_path = get_config_path()

In [17]:
from hydrate.utils import DotConfig
config = DotConfig(config_path)

In [24]:
from databricks.connect import DatabricksSession as SparkSession
spark = SparkSession.builder.serverless(True).getOrCreate()

In [25]:
spark.createDataFrame(clean_data.head(100)).write.mode('overwrite').option('mergeSchema', 'true').saveAsTable(f'{config.catalog}.{config.schema}.{config.process.table}')

AnalysisException: [DELTA_FAILED_TO_MERGE_FIELDS] Failed to merge fields 'state' and 'state'

JVM stacktrace:
com.databricks.sql.transaction.tahoe.DeltaAnalysisException
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.$anonfun$mergeDataTypes$1(SchemaMergingUtils.scala:215)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.mutable.ArrayOps$ofRef.map(ArrayOps.scala:198)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.merge$1(SchemaMergingUtils.scala:204)
	at com.databricks.sql.transaction.tahoe.schema.SchemaMergingUtils$.mergeDataTypes(Sc